# 『pytorch ＆ 深層学習プログラミング』　備忘録chap2


## 目次
- [何故スカラーなのか（p83）](#何故スカラーなのか（p83）)
- [何故スカラーにするためにsumを用いるのか（p83）](#何故スカラーにするためにsumを用いるのか（p83）)
- [failed to execute WindowsPath（'dot'）, make sure the Graphviz executables are on your systems' PATHの解決（p82）](#failedtoexecuteWindowsPath（'dot'）,makesuretheGraphvizexecutablesareonyoursystems'PATHの解決（p82）)

協力：K.I.先生

<br>
<br>
<br>

## 何故スカラーなのか（p83）
---
<br>
pytorchでは、勾配計算をする際に対象の関数がスカラーである必要があるらしい。

正直、「スカラーにしてしまったら元の関数がわからなくなるじゃないか！」と思ってしまったのだが、どうやら**pytorchはそのスカラーがどのようにして生成されたのかを記憶している**ようである。

pytorchの仕様として基本的にスカラーにしないと勾配計算ができないので、ここは黙ってスカラーにしておくのが無難である。
<br>
<br>
<br>


## 何故スカラーにするためにsumを用いるのか（p83）
---
<br>
スカラーにしてもその生前の記憶は残っていると分かったものの、次に生じた疑問は

「スカラーにするだけならsum()以外の関数でも良いのでは？」

というものである。

試しにmean()でスカラーにしてからbackward()をかけると、結果は微分の値にはならなかった。つまり、sum()を用いてtensorの合計を求めるという計算には何か重要な意味があるはずだと分かる。

図．mean関数を使ったときのグラフ

![](image/2022-06-27-19-48-33.png)

図．sum関数を使ったときのグラフ

![](image/2022-06-27-19-52-51.png)

両者を見比べると、何かが分かるような気がする。が、分からない。

<br>
<br>

### ▪ 理解のための前提知識
<br>

それを理解するためには、以下の事を知っておかなければならない。
<br>

- xという「ベクトル」に対しても、x.backward() をする時に、（）に引数を入れると計算ができるということ
- その引数は、xと同じサイズのベクターにするということ
  
<br>
である。例えばこのテキストでは、

z = y.sum() 　→ 　z.backward() 

としているが、その代わりにいきなり y.backward(torch.ones(y.size())) としてもよく、実際同じ結果になる。（ y.sum() はしない）

ここで torch.ones(y.size()) は　y と同じ大きさで、全ての要素が１であるベクトルである。この計算は「全ての y を同等の weight で扱って x.grad を計算する」という意味になっている。


<br>
<br>

###  ▪ z.backward()に隠された引数１
<br>
もうお気づきかもしれないが、実は z.backward() のカッコの中には 1 が隠されている。これは z を大きさ 1 のスカラーにしているので考えてみれば当然であるが、それに気づくとsum()の意味が分かってくる。

y.sum()を行うことで**どんなtensorであっても確実に大きさ１のスカラーに変換し、同時に均等のweightで全ての要素を足し合わせたことになる**のである！



<br>
<br>

###  ▪ 均等のweightによる足し算
<br>
最初はこれに突っかかってしまったのだが、それぞれの要素に何も掛け算せず（×0.8 , ×1.3 等）足し合わせているのだから、重みは当然1である。全ての重みを１にすると、微分することと同義となるのだ。

しかし考えてみると、機械学習やディープラーニングにおいて全ての重みを1にするなど意味のない行為である。今後、この重みを調節することによって高度な学習をするようになるのだろうか、と期待しつつ今後勉強していきたい。

<br>
<br>
<br>

## failed to execute WindowsPath（'dot'）, make sure the Graphviz executables are on your systems' PATHの解決（p82）
---
<br>
なんと、久々に82ページのコードを実行しようとすると、上記の様なエラーが出てしまった。このエラーを解決するためには、

1．仮想環境の作成　→　2．graphvizのインストール　→　3.環境設定でPATHの新規作成

という3段階を経なければならない。なんだかんだ4時間以上かかってしまった。

![](image/2022-06-27-15-50-36.png)

エラー内容
：
…**failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH**




### 1.仮想環境の作成
<br>
VScodeにおいて仮想環境を作る場合、ターミナルをVScode内で開くことができるので少し楽だ。手順は以下に示す。

- 仮想環境を作成する適当なフォルダを作成する。(例．\Users\username\imaginary_environment\graphviz ディレクトリを作る)
- VScodeにて｢F1｣を押し、｢Create Terminal｣を選択し、ターミナルを開く
- (windowsの場合)｢py -3 -m venv .venv｣を作成したディレクトリで実行する
- ターミナルを一度閉じ、再度開く。自分の場合ExecutionPolicyのエラーが出てしまったため、｢Set-ExecutionPolicy RemoteSigned -Scope Process｣をターミナルで毎度入力し、権限を一時的に許可している

これで仮想環境の作成は完了である。あとは必要な物をインストールすれば良い

### 2.graphvizのインストール
<br>
ここで落とし穴があり、py -m pip install torchvizをターミナルで行ったとしても**インストールしてあるgraphvizとの繋がりが形成されるだけで、graphviz自体はインストールされない**という事だ。

つまり、graphvizは公式サイトから自分でインストールしなければならない。

### 3.環境設定でPATHの新規作成
<br>
インストールしただけでは、実はまだエラーは解消しない。エラー文をよく読むと、PATHが何たらと書いてある。これが最後の問題点である。

自分も良く分かっていないが、PATHとは探し物をする際の道しるべのようなもので、ここに登録しておいた道（PATH）はコンピューターが毎度歩いて探し物をしてくれるらしい。

コントロールパネルから「システム」を選択しシステムの詳細設定、そして「環境変数」を選択する。

そこでユーザーの環境変数において新規を選択し、変数名にはPATH、変数値にはGraphviz\binが収納してあるディレクトリを入力すれば、ようやくエラーの解消である。